In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import StandardScaler
import re
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV


training_path = '/home/colinprice/Documents/Maize_GxE_Competition/Maize_GxE_Competition_Data_2024_UPDATE/Training_data/'

trait_data = pd.read_csv(training_path+"1_Training_Trait_Data_2014_2023.csv")

meta_data = pd.read_csv(training_path+"2_Training_Meta_Data_2014_2023.csv")

soil_data = pd.read_csv(training_path+"3_Training_Soil_Data_2015_2023.csv")

In [3]:
EC_data = pd.read_csv(training_path+"6_Training_EC_Data_2014_2023.csv")

def find_common_columns(*dataframes):
    """
    This function takes multiple pandas DataFrame objects as input and returns a list
    of columns that are common to all dataframes.
    """
    # Convert the column names of the first dataframe to a set
    common_columns = set(dataframes[0].columns)
    
    # Find the intersection with the column names of the other dataframes
    for df in dataframes[1:]:
        common_columns.intersection_update(df.columns)
    
    return list(common_columns)

# Example usage with your loaded dataframes
common_columns = find_common_columns(trait_data, meta_data, soil_data, EC_data)

print("Common columns across all dataframes:", common_columns)
# Just ENV
# Merge Trait Data and 


combined_data = pd.merge(trait_data, meta_data, on='Env', how='inner')
combined_data = pd.merge(combined_data, soil_data, on='Env', how='inner')
combined_data = pd.merge(combined_data, EC_data, on='Env', how='inner')

# Picking Lower Left as the coordinate to base things on arbitrarily
columns_to_drop = [
    'Year_y', 'City', 'Experiment_Code', 'Farm', 'Field', 
    'Trial_ID (Assigned by collaborator for internal reference)', 
    'Soil_Taxonomic_ID and horizon description, if known', 
    'Weather_Station_Serial_Number (Last four digits, e.g. m2700s#####)', 
    'Pre-plant_tillage_method(s)', 'In-season_tillage_method(s)', 
    'Type_of_planter (fluted cone; belt cone; air planter)', 
    'System_Determining_Moisture', 'Pounds_Needed_Grain_Moisture', 
    'Issue/comment_#1', 'Issue/comment_#2', 'Issue/comment_#3', 
    'Issue/comment_#4', 'Issue/comment_#5', 'Issue/comment_#6', 
    'Issue/comment_#7', 'Issue/comment_#8', 'Cardinal_Heading_Pass_1', 
    'LabID', 'Texture', 'Comments',
    'Latitude_of_Field_Corner_#2 (lower right)',
    'Longitude_of_Field_Corner_#2 (lower right)',
    'Latitude_of_Field_Corner_#3 (upper right)',
    'Longitude_of_Field_Corner_#3 (upper right)',
    'Latitude_of_Field_Corner_#4 (upper left)',
    'Longitude_of_Field_Corner_#4 (upper left)',
    'Hybrid_orig_name',
    'Range',
    'Pass',
    'Plot_Area_ha',
    'Pounds_Needed_Soil_Moisture'
]

# Drop the columns from the dataframe, modifying it in place
combined_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')
print(combined_data.head())

date_columns = ['Date_Planted', 'Date_Harvested', 
                'Date_weather_station_placed', 'Date_weather_station_removed',
                'Date Received', 'Date Reported']

for date_col in date_columns:
    combined_data[date_col] = pd.to_datetime(combined_data[date_col], errors='coerce')

# Calculate the differences in days between the relevant columns
combined_data['Days_to_Harvest'] = (combined_data['Date_Harvested'] - combined_data['Date_Planted']).dt.days
combined_data['Days_Weather_Station'] = (combined_data['Date_weather_station_removed'] - combined_data['Date_weather_station_placed']).dt.days
combined_data['Days_to_Report'] = (combined_data['Date Reported'] - combined_data['Date Received']).dt.days

combined_data.rename(columns={'Year_x': 'Year'}, inplace=True)

# Drop the original date columns
columns_to_drop = ['Date_Planted', 'Date_Harvested', 
                   'Date_weather_station_placed', 'Date_weather_station_removed',
                   'Date Received', 'Date Reported']

combined_data.drop(columns=columns_to_drop, inplace=True)

# Display the updated DataFrame to verify changes
print(combined_data.head())

print("Number of rows in the combined dataframe:", combined_data.shape[0])


columns_to_onehot_encode = [
    'Treatment', 'Previous_Crop', 'Irrigated']

onehot_encoded_data = pd.get_dummies(combined_data[columns_to_onehot_encode], columns=columns_to_onehot_encode)
combined_data.drop(columns=columns_to_onehot_encode, inplace=True)
combined_data = pd.concat([combined_data, onehot_encoded_data], axis=1)

# Drop to not learn over
columns_to_not_learn_over = [
    'Env', 'Year', 'Field_Location', 'Experiment', 'Replicate', 'Block', 'Plot', 'Hybrid', 'Hybrid_Parent1',
    'Hybrid_Parent2']

combined_data = combined_data.drop(columns=columns_to_not_learn_over, inplace=False)
combined_data.fillna(0, inplace=True)

y = combined_data['Yield_Mg_ha'] # Target variable
combined_data.drop(columns='Yield_Mg_ha', inplace=True)
scaler = StandardScaler()

onehot_encoded_data_columns = set(onehot_encoded_data.columns)
combined_data_columns = set(combined_data.columns)

remaining_columns = combined_data_columns - onehot_encoded_data_columns
remaining_columns_list = list(remaining_columns)

combined_data[remaining_columns_list] = scaler.fit_transform(combined_data[remaining_columns_list])

# drop the trait data we wont have for test set
columns_to_drop_from_trait = ['Stand_Count_plants', 'Pollen_DAP_days', 'Silk_DAP_days', 'Plant_Height_cm', 'Ear_Height_cm', 'Root_Lodging_plants', 'Stalk_Lodging_plants', 'Grain_Moisture', 'Twt_kg_m3']
combined_data.drop(columns=columns_to_drop_from_trait, inplace=True)
X = combined_data  # Features


Common columns across all dataframes: ['Env']
         Env  Year_x Field_Location  ...     LL__8     LL__9    LL__10
0  IAH1_2015    2015           IAH1  ...  0.090938  0.090938  0.090938
1  IAH1_2015    2015           IAH1  ...  0.090938  0.090938  0.090938
2  IAH1_2015    2015           IAH1  ...  0.090938  0.090938  0.090938
3  IAH1_2015    2015           IAH1  ...  0.090938  0.090938  0.090938
4  IAH1_2015    2015           IAH1  ...  0.090938  0.090938  0.090938

[5 rows x 717 columns]
         Env  Year  ... Days_Weather_Station Days_to_Report
0  IAH1_2015  2015  ...                  NaN            NaN
1  IAH1_2015  2015  ...                  NaN            NaN
2  IAH1_2015  2015  ...                  NaN            NaN
3  IAH1_2015  2015  ...                  NaN            NaN
4  IAH1_2015  2015  ...                  NaN            NaN

[5 rows x 714 columns]
Number of rows in the combined dataframe: 114915


/tmp/ipykernel_362512/1658712467.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_data[date_col] = pd.to_datetime(combined_data[date_col], errors='coerce')
/tmp/ipykernel_362512/1658712467.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_data[date_col] = pd.to_datetime(combined_data[date_col], errors='coerce')


In [4]:
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)  # You can adjust the number of trees

# Setup 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

mse_scores = []
correlation_scores = []

# Lists to store actual and predicted values for later analysis
all_actuals = []
all_predictions = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Store predictions and actuals
    all_actuals.extend(y_test)
    all_predictions.extend(y_pred)
    
    # Calculate MSE
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)
    
    # Calculate Pearson correlation coefficient
    r, _ = pearsonr(y_test, y_pred)
    correlation_scores.append(r)

# Calculate and print overall performance
overall_mse = np.mean(mse_scores)
overall_r = np.mean(correlation_scores)
print(f'Overall MSE: {overall_mse}')
print(f'Overall Pearson Correlation Coefficient: {overall_r}')

# Print per-split performance
for i, (mse, r) in enumerate(zip(mse_scores, correlation_scores), 1):
    print(f'Split {i} - MSE: {mse}, Pearson r: {r}')


Overall MSE: 7.763537899631382
Overall Pearson Correlation Coefficient: 0.6630475691508398
Split 1 - MSE: 7.698546694087034, Pearson r: 0.6632540812219363
Split 2 - MSE: 7.905023986808943, Pearson r: 0.6585590535385055
Split 3 - MSE: 7.561469337071198, Pearson r: 0.6690791903299977
Split 4 - MSE: 7.734753570675763, Pearson r: 0.6640056404276505
Split 5 - MSE: 7.917895909513966, Pearson r: 0.6603398802361093


In [5]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
                           param_grid=param_grid,
                           cv=kf,
                           scoring='neg_mean_squared_error',  # MSE scoring
                           n_jobs=-1,
                           verbose=2)
grid_search.fit(X, y)

best_model = grid_search.best_estimator_

# Predict using the best model
y_pred = best_model.predict(X)  # Assuming you want to predict the entire set or use a train/test split as needed

# Calculate MSE and Pearson correlation
mse = mean_squared_error(y, y_pred)
r, _ = pearsonr(y, y_pred)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best MSE: {mse}")
print(f"Pearson Correlation Coefficient: {r}")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/home/colinprice/miniconda3/envs/maize/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
46 fits failed with the following error:
Traceback (most recent call last):
  File "/home/colinprice/miniconda3/envs/maize/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/colinprice/miniconda3/envs/maize/lib/python3.10/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/home/colinprice/miniconda3/envs/maize/lib/python3.10/site-packages/sklearn/base.py", line 436, in _v

Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 100}
Best MSE: 7.72076520768308
Pearson Correlation Coefficient: 0.6653537206838245
